# The Full Dataset
## Part 1: Data Extraction, GMM Fitting, and Visualization

This part focuses on loading the event log, converting it into a Pandas DataFrame, and extracting the "hour of day" from timestamps. A Gaussian Mixture Model (GMM) is then fitted to this extracted feature to identify patterns in the time-of-day data. The best GMM is selected based on the Bayesian Information Criterion (BIC), and its fit along with individual components are visualized.

## Implementation Steps:

1. **Load Data**: Import the BPI 2017 event log and convert it to a DataFrame.
2. **Extract Features**: Extract the "hour of day" from the timestamp.
3. **Fit GMM**: Fit a GMM for a range of components and select the model with the lowest BIC.
4. **Visualization**: Plot the density estimation and individual components of the best GMM.
5. **BIC Score**: Print the BIC score of the optimal GMM to evaluate its fit.


In [ ]:
import pm4py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.stats import norm

# Load BPI Challenge 2017 data
BPI2017 = "./BPI Challenge 2017 - Offer log.xes"
event_log = pm4py.read_xes(BPI2017)
data = pm4py.convert_to_dataframe(event_log)

# Extract hour of day from timestamps
data['time_of_day'] = data['time:timestamp'].dt.hour + data['time:timestamp'].dt.minute / 60
master_data = data
# Prepare data for GMM fitting
X = data['time_of_day'].values.reshape(-1, 1)

# Fit GMM and find the best model based on BIC
lowest_bic = np.infty
best_gmm = None
bic_list = []
n_components_range = range(1, 11)

for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0)
    gmm.fit(X)
    bic = gmm.bic(X)
    bic_list.append(bic)
    if bic < lowest_bic:
        lowest_bic, best_gmm = bic, gmm
    
# Show the best GMM
print(f"The best GMM has {best_gmm.n_components} components with a BIC score of {lowest_bic:.2f}.")
# Number of bins for the histogram
bins = 24

# Histogram data
hist_data, bin_edges = np.histogram(X.flatten(), bins=bins, density=True)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Plot the histogram of the hour of day and the density estimated by the best GMM
plt.figure()
plt.hist(X.flatten(), bins=24, density=True, alpha=0.6, label='Data Histogram')
x = np.linspace(X.min(), X.max(), 10000).reshape(-1, 1)
logprob = best_gmm.score_samples(x)


# Plot each individual GMM component
for i in range(best_gmm.n_components):
    weight = best_gmm.weights_[i]
    mean = best_gmm.means_[i, 0]
    covariance = best_gmm.covariances_[i, 0, 0]
    individual_pdf = weight * norm.pdf(x, mean, np.sqrt(covariance))
    plt.plot(x, individual_pdf, '--', label=f'Component {i+1}')

# GMM parameters
gmm_means = best_gmm.means_.flatten()
gmm_variances = best_gmm.covariances_.flatten()
gmm_weights = best_gmm.weights_.flatten()

# Now, print or store the values you need
print("Histogram Bin Centers:", bin_centers)
print("Histogram Bin Heights:", hist_data)
print("Bin Edges:", bin_edges)
print("GMM Means:", gmm_means)
print("GMM Variances:", gmm_variances)
print("GMM Weights:", gmm_weights)

pdf = np.exp(logprob)
plt.plot(x, pdf, '-k', label='GMM Fit')
plt.title('Hour of Day and GMM Density Estimation')
plt.xlabel('Hour of Day')
plt.ylabel('Density')
plt.legend()
plt.show()

# The Segmented Dataset

## Part 2: Detailed Exploration through Recursive Partitioning

After initial assessments with the whole dataset, we delve deeper by dissecting the data into more homogenous segments. This step employs recursive partitioning, focusing on pivotal features to identify subsets that share common characteristics. Each subset is then modeled using Gaussian Mixture Models (GMM) to grasp the underlying distribution of events over time. This intricate process allows for a nuanced understanding of the data's structure and variability.

### Implementation Steps:

1. **Feature Identification using Decision Trees**: Utilize a Decision Tree Regressor to pinpoint the most influential feature that impacts the timing of events. This method ensures that the partitioning is guided by data-driven insights, leading to meaningful segmentation.

2. **Recursive Partitioning**: Systematically divide the dataset based on the identified feature. This process is recursive, meaning it repeats this division within each resulting subset until a specified depth or minimum subset size is reached. This iterative approach ensures that even the most granular patterns can be uncovered.

3. **Gaussian Mixture Modeling**: For each subset created through partitioning, a GMM is fitted. This step is crucial for understanding how events are distributed over time within each segment. By modeling these distributions, we can identify unique patterns that were not apparent in the aggregated data.

4. **Visual Analysis**: Select subsets are visualized to showcase their GMM fits, providing a graphical representation of the time-based event distribution and the effectiveness of the segmentation.

5. **Averaging Metrics**: Aggregate measures, including the average BIC score and the average number of components across all subsets, are computed. These metrics offer insights into the overall complexity and homogeneity of the data after segmentation. Lower BIC scores and fewer components suggest that the recursive partitioning has successfully identified more uniform groups within the data.

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import random

# Function to identify the most important feature
def plot_subset_gmm_with_components(current_data, subset_index):
    subset_conditions, subset_data = final_partitions[subset_index]
    conditions_str = ', '.join(subset_conditions) if subset_conditions else "No conditions (initial dataset)"
    print(f"Conditions for subset {subset_index}: {conditions_str}")

    # Prepare the data for GMM fitting
    X = subset_data['time_of_day'].values.reshape(-1, 1)

    if len(X) > 1:  # Ensure there are enough samples for GMM fitting
        n_components_range = range(1, min(len(X), 11))
        best_gmm = None
        lowest_bic = np.infty
        for n_components in n_components_range:
            gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0).fit(X)
            bic = gmm.bic(X)
            if bic < lowest_bic:
                lowest_bic = bic
                best_gmm = gmm

        # Plotting the overall GMM fit
        plt.hist(X.flatten(), bins=24, density=True, alpha=0.6, color='skyblue', label='Data Histogram')
        x = np.linspace(X.min(), X.max(), 1000).reshape(-1, 1)
        logprob = best_gmm.score_samples(x)
        pdf = np.exp(logprob)
        plt.plot(x, pdf, '-k', label='Overall GMM Fit')

        # Plotting each component
        for i in range(best_gmm.n_components):
            mean = best_gmm.means_[i][0]
            cov = best_gmm.covariances_[i][0][0]
            component_pdf = np.exp(-0.5 * (x - mean) ** 2 / cov) / np.sqrt(2 * np.pi * cov) * best_gmm.weights_[i]
            plt.plot(x, component_pdf, '--', label=f'Component {i+1}')

        plt.title(f'Subset {subset_index}: Time of Day and GMM Density Estimation')
        plt.xlabel('Time of Day')
        plt.ylabel('Density')
        plt.legend()
        plt.show()

        # Additional Information
        # Histogram Data
        hist_data, bin_edges = np.histogram(X, bins=24, density=True)
        bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

        # GMM Parameters
        gmm_means = best_gmm.means_.flatten()
        gmm_variances = best_gmm.covariances_.flatten()
        gmm_weights = best_gmm.weights_.flatten()

        print("Histogram Bin Centers:", bin_centers)
        print("Histogram Bin Heights:", hist_data)
        print("Bin Edges:", bin_edges)
        print("GMM Means:", gmm_means)
        print("GMM Variances:", gmm_variances)
        print("GMM Weights:", gmm_weights)

    else:
        print("Not enough data for GMM fitting.")
        
def train_decision_tree_and_identify_important_feature_optimized(data, target_column_name):
    data_copy = data.copy()
    if not np.issubdtype(data_copy[target_column_name].dtype, np.number):
        data_copy[target_column_name] = pd.to_numeric(data_copy[target_column_name], errors='coerce')
    data_copy = data_copy.dropna(subset=[target_column_name])

    categorical_cols = data_copy.select_dtypes(include=['object', 'bool']).columns
    le = LabelEncoder()
    for column in categorical_cols:
        data_copy[column] = le.fit_transform(data_copy[column].astype(str))

    cols_to_drop = ['case:ApplicationID', 'EventID', 'time:timestamp']
    data_copy = data_copy.drop(columns=cols_to_drop, errors='ignore')

    X = data_copy.drop(columns=[target_column_name])
    y = data_copy[target_column_name].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    tree_reg = DecisionTreeRegressor(random_state=42)
    tree_reg.fit(X_train, y_train)

    feature_importances = tree_reg.feature_importances_
    most_important_feature_index = np.argmax(feature_importances)
    most_important_feature_name = X.columns[most_important_feature_index]

    return most_important_feature_name

def segment_data_based_on_feature(current_data, feature_name):
    segmented_data = {}
    feature_values = current_data[feature_name].values

    if np.issubdtype(current_data[feature_name].dtype, np.number):
        unique_values = np.unique(feature_values)
        if len(unique_values) > 10:  # Adjust this threshold as needed
            feature_values_reshaped = feature_values.reshape(-1, 1)
            n_components_range = range(1, min(len(unique_values), 11))
            models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(feature_values_reshaped) for n in n_components_range]
            best_gmm = min(models, key=lambda model: model.bic(feature_values_reshaped))
            clusters = best_gmm.predict(feature_values_reshaped)
            current_data['cluster'] = clusters
            for cluster in np.unique(clusters):
                cluster_data = current_data[current_data['cluster'] == cluster]
                segmented_data[f'Cluster {cluster}'] = cluster_data.drop(columns=['cluster'])
        else:
            for value in unique_values:
                segmented_data[f'{feature_name}={value}'] = current_data[current_data[feature_name] == value]
    else:
        unique_values = current_data[feature_name].unique()
        for value in unique_values:
            segmented_data[f'{feature_name}={value}'] = current_data[current_data[feature_name] == value]

    return segmented_data

def recursive_partition(current_data, excluded_features, threshold, max_depth, current_depth=0, conditions=[]):
    if current_depth >= max_depth or len(current_data) <= threshold:
        return [(conditions, current_data)]

    features = [col for col in current_data.columns if col not in excluded_features + ['time_of_day']]
    data_for_tree = current_data[features + ['time_of_day']].dropna()
    if data_for_tree.empty or len(features) == 0:
        return [(conditions, current_data)]

    important_feature = train_decision_tree_and_identify_important_feature_optimized(data_for_tree, 'time_of_day')
    if important_feature not in excluded_features:
        partitions = segment_data_based_on_feature(current_data, important_feature)
        results = []
        for segment_key, segment_data in partitions.items():
            new_conditions = conditions + [segment_key]
            results += recursive_partition(segment_data, excluded_features + [important_feature], threshold, max_depth, current_depth + 1, new_conditions)
        return results
    else:
        return [(conditions, current_data)]

# Function to calculate averages
def calculate_averages(final_partitions):
    total_components = 0
    total_bic = 0
    for _, subset_data in final_partitions:
        X = subset_data['time_of_day'].values.reshape(-1, 1)
        if len(X) > 1:
            n_components_range = range(1, min(len(X), 11))
            lowest_bic = np.infty
            best_n_components = None
            for n_components in n_components_range:
                gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0).fit(X)
                bic = gmm.bic(X)
                if bic < lowest_bic:
                    lowest_bic = bic
                    best_n_components = n_components
            total_components += best_n_components
            total_bic += lowest_bic
    average_components = total_components / len(final_partitions)
    average_bic = total_bic / len(final_partitions)
    return average_components, average_bic


excluded_features = ['case:ApplicationID', 'OfferID', 'EventID', 'EventOrigin', 'case:concept:name', 'cluster', 'time:timestamp']
final_partitions = recursive_partition(data.copy(), excluded_features, 100, 4)
average_components, average_bic_total_subsets = calculate_averages(final_partitions)
print(final_partitions)
print(f"Average number of GMM components: {average_components}")
print(f"Average BIC score: {average_bic_total_subsets}")

# Subsets that are modelled with one normal distribution component
## Part 3: Single-Component Optimal Fit Filtering and Averages Calculation

In this section, the aim is to refine our analysis by focusing on subsets where a Gaussian Mixture Model (GMM) with a single component is deemed optimal. This constraint allows for a more targeted examination of data segments that exhibit a relatively uniform distribution of event times throughout the day.

## Steps:

1. **Filtering**: Exclude subsets that are best modeled with more than one GMM component.
2. **Average BIC Calculation**: Calculate the average Bayesian Information Criterion (BIC) score across the filtered subsets to evaluate the fit's quality.
3. **Interpretation**: The resulting average provides insights into how well a simple model can represent the data, offering a perspective on the underlying process complexity.


In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import random

# Function to filter subsets optimally modeled by a single GMM component and calculate their BIC scores
def filter_and_calculate_bic_single_component_subsets(final_partitions):
    bic_scores = []
    single_component_subsets = []

    for partition in final_partitions:
        _, subset_data = partition
        X = subset_data['time_of_day'].values.reshape(-1, 1)
        if len(X) > 1:
            best_bic = np.inf
            best_model = None
            for n_components in range(1, min(11, len(X) + 1)):
                gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0).fit(X)
                bic = gmm.bic(X)
                if bic < best_bic:
                    best_bic = bic
                    best_model = gmm
            # Check if the best model for this subset has exactly one component
            if best_model.n_components == 1:
                bic_scores.append(best_bic)
                single_component_subsets.append((best_model, X))

    return bic_scores, single_component_subsets

# Assuming final_partitions is already defined from the segmentation step
bic_scores, single_component_subsets = filter_and_calculate_bic_single_component_subsets(final_partitions)

# Calculate and print the average BIC score across subsets optimally modeled with a single component
if bic_scores:
    average_bic_single_component = np.mean(bic_scores)
    print(f"Average BIC score for subsets best modeled with a single GMM component: {average_bic_single_component:.2f}")
    print(f"Number of subsets with a single component optimal: {len(bic_scores)}")
else:
    print("No subsets are best modeled with a single GMM component.")

# Plotting one of the subsets randomly if available
if single_component_subsets:
    best_model, X = single_component_subsets[7]
    plt.hist(X.flatten(), bins=24, density=True, alpha=0.6, color='skyblue', label='Data Histogram')
    x = np.linspace(X.min(), X.max(), 1000).reshape(-1, 1)
    logprob = best_model.score_samples(x)
    pdf = np.exp(logprob)
    plt.plot(x, pdf, '-k', label='Optimal Single Component GMM Fit')

    # Since it's a single component, extract its mean and covariance for plotting
    mean = best_model.means_[0]
    covariance = best_model.covariances_[0]
    component_pdf = np.exp(-0.5 * (x - mean) ** 2 / covariance) / np.sqrt(2 * np.pi * covariance)
    plt.plot(x, component_pdf, '--r', label='GMM Component')

    plt.title("Random Subset with Optimal Single Component GMM")
    plt.xlabel("Time of Day")
    plt.ylabel("Density")
    plt.legend()
    plt.show()

    # Additional Information
    # Histogram Data
    hist_data, bin_edges = np.histogram(X, bins=24, density=True)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    # GMM Parameters
    gmm_means = best_model.means_.flatten()
    gmm_variances = best_model.covariances_.flatten()
    gmm_weights = best_model.weights_.flatten()

    print("Histogram Bin Centers:", bin_centers)
    print("Histogram Bin Heights:", hist_data)
    print("Bin Edges:", bin_edges)
    print("GMM Means:", gmm_means)
    print("GMM Variances:", gmm_variances)
    print("GMM Weights:", gmm_weights)
else:
    print("No suitable subsets for plotting.")

# Comparing the Models
## Part 4: Comparison of BIC Scores Across Different Models
In this part, we compare the Bayesian Information Criterion (BIC) scores obtained from three different modeling approaches to understand their effectiveness in capturing the underlying distribution of the data:

1. **Overall Data Model**: The BIC score for the entire dataset modeled with a Gaussian Mixture Model (GMM) to find the best fit.
2. **Segmented Subsets Model**: After segmenting the data based on significant features, we model each subset with GMMs and calculate the average BIC score across all subsets.
3. **Single-Component Subsets Model**: Focusing on subsets that are optimally modeled with a single component GMM, we calculate the average BIC score for these subsets.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Labels for each category
categories = ['Overall Data', 'Segmented Subsets', 'Single-Component-Optimal Subsets']
categories_comparison = ['Segmented Subsets', 'Single-Component-Optimal Subsets']

# BIC scores for each category
bic_scores = [lowest_bic, average_bic_total_subsets, average_bic_single_component]
bic_scores_comparison = [average_bic_total_subsets, average_bic_single_component]

# Creating the bar chart with a logarithmic scale
plt.figure(figsize=(10, 6))
plt.bar(categories, bic_scores, color=['blue', 'green', 'red'])
plt.yscale('log')  # Applying logarithmic scale

# Adding the title and labels
plt.title('Comparison of BIC Scores Across Different Models (Log Scale)')
plt.ylabel('BIC Score (log scale)')
plt.xlabel('Model Type')

# Showing the plot with log scale
plt.tight_layout()
plt.show()

# Creating a separate bar chart with a normal scale for segmented vs single-component
plt.figure(figsize=(10, 6))
plt.bar(categories_comparison, bic_scores_comparison, color=['green', 'red'])

# Adding the title and labels for the comparison
plt.title('Comparison of BIC Scores: Segmented vs Single-Component')
plt.ylabel('BIC Score')
plt.xlabel('Model Type')

# Showing the plot with normal scale
plt.tight_layout()
plt.show()

import matplotlib.pyplot as plt

# Number of subsets
total_subsets_count = len(final_partitions)  # Assuming 'final_partitions' contains all subsets
single_component_optimal_count = len(single_component_subsets)  # Assuming 'single_component_subsets' contains only those optimal for a single component

# Calculating fractions
fractions = [single_component_optimal_count, total_subsets_count - single_component_optimal_count]
labels = ['Single-Component-Optimal Subsets', 'Other Subsets']

# Creating the pie chart
plt.figure(figsize=(8, 8))
plt.pie(fractions, labels=labels, autopct='%1.1f%%', startangle=140, colors=['red', 'grey'])

plt.title('Fraction of Single-Component-Optimal Subsets')
plt.tight_layout()
plt.show()


# Hyperparameter Tuning

## Optimizing Recursive Partitioning and GMM Fitting

To fine-tune the segmentation and modeling of our dataset, we employ a methodical approach to identify the optimal parameters for recursive partitioning and Gaussian Mixture Model (GMM) fitting. Specifically, we aim to determine the best values for the **threshold** for partitioning and the **maximum recursion depth**. These parameters significantly influence the granularity of segmentation and, consequently, the fitting of GMMs to these segments.

### Implementation Steps:

1. **Parameter Variation**: We iterate over a predefined range of values for both the threshold and max_depth parameters. The threshold controls the minimum size a segment must have to be considered for further partitioning, while max_depth limits the recursion level to prevent over-segmentation.
   
2. **Evaluation with Average BIC**: For each parameter configuration, we use the recursive partitioning function to segment the dataset accordingly. Then, we fit GMMs to each segment and calculate the Bayesian Information Criterion (BIC) score for these models. The BIC score helps us evaluate the model fit while penalizing over-complexity. We calculate the average BIC score across all segments for each parameter setup.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Assuming data is prepared and final_partitions is generated correctly

# Define ranges of parameters to test
thresholds = [10, 50, 100, 500, 1000]  # Example threshold values
depths = [1, 2, 4, 8]  # Example max_depth values

# Prepare to store results
results = []

# Nested loop over both thresholds and depths
for threshold in thresholds:
    for depth in depths:
        final_partitions = recursive_partition(data.copy(), excluded_features, threshold, depth)
        _, average_bic = calculate_averages(final_partitions)
        results.append((threshold, depth, average_bic))

# Extracting results for plotting
thresholds, depths, bics = zip(*results)
print(bics)
# Normalize BIC values for point sizes, ensure division by zero is handled
bic_min = min(bics)
bic_max = max(bics)
sizes = [20 + 180 * (bic - bic_min) / (bic_max - bic_min) if bic_max - bic_min > 0 else 100 for bic in bics]

# Find optimal combination (minimal BIC)
optimal_index = np.argmin(bics)

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for all combinations
ax.scatter(thresholds, depths, bics, s=sizes, c='blue', alpha=0.6)

# Highlight the optimal combination
ax.scatter([thresholds[optimal_index]], [depths[optimal_index]], [bics[optimal_index]], s=[sizes[optimal_index]], c='red', alpha=1, edgecolor='black', label='Optimal BIC')

# Adding labels and title
ax.set_xlabel('Threshold')
ax.set_ylabel('Max Depth')
ax.set_zlabel('Average BIC')
ax.set_title('Average BIC vs Threshold vs Max Depth')
ax.legend()

plt.show()

# Part 5: Hyperparameter Tuning and Complexity Analysis

In this part of the analysis, we embark on a meticulous journey to refine our partitioning strategy by tuning the hyperparameters and evaluating the complexity of our recursive partitioning algorithm. This process is crucial for optimizing the performance of our model and ensuring that we strike a perfect balance between model complexity and interpretability.

## Hyperparameter Tuning

To fine-tune the parameters of our recursive partitioning process, we explore a range of values for two critical hyperparameters: the `threshold` for the minimum subset size and the `max_depth` of recursive partitioning. The objective is to identify the optimal combination of these parameters that minimizes the Bayesian Information Criterion (BIC) score, indicating a better model fit with lower complexity.

### Implementation Steps:

1. We conduct a comprehensive grid search over specified ranges of `threshold` and `max_depth` values.
2. For each combination of parameters, we execute the recursive partitioning function to segment the dataset and then fit Gaussian Mixture Models (GMM) to each subset.
3. We calculate the average BIC score across all partitions for each parameter combination, aiming to find the set of parameters that yield the lowest average BIC.

### Results

A 3D scatter plot visualizes the outcomes of this hyperparameter tuning process, with each point representing a combination of `threshold` and `max_depth` values. The size of each point correlates with the BIC score, where larger points signify higher BIC values. The optimal combination, denoted by a distinct color, highlights the parameters that achieve the lowest average BIC, signifying the most efficient partitioning strategy.

## Complexity Analysis

Following the hyperparameter tuning, we delve into the complexity analysis of our recursive partitioning approach. This analysis aims to understand how the execution time of our partitioning algorithm varies with different `threshold` and `max_depth` values, providing insights into the scalability and efficiency of our method.

### Implementation Steps:

1. We measure the execution time for varying `threshold` values while keeping the `max_depth` constant and vice versa.
2. This procedure allows us to observe the impact of each hyperparameter on the algorithm's computational demands.

### Results

Two line plots showcase the execution times against varying `threshold` and `max_depth` values. These plots reveal trends in how the complexity of our partitioning approach scales with the size of the subsets and the depth of recursion. A thorough understanding of these dynamics assists in making informed decisions when adjusting hyperparameters to balance model performance and computational efficiency.


In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

# Assuming 'data' is your DataFrame prepared for the analysis

# Define the range of values for threshold and max depth
threshold_values = np.arange(1000, 10001, 2000)  # Example: from 100 to 1000 in steps of 100
max_depth_values = np.arange(1, 6, 1)  # Example: from 1 to 10 in steps of 1

# Lists to store execution times
times_threshold = []
times_depth = []

# Measure time for varying thresholds
for threshold in threshold_values:
    start_time = time.time()
    final_partitions = recursive_partition(data.copy(), excluded_features, threshold, 5)  # Use a default depth value that makes sense for your dataset
    calculate_averages(final_partitions)
    end_time = time.time()
    times_threshold.append(end_time - start_time)

# Measure time for varying max depths
for max_depth in max_depth_values:
    start_time = time.time()
    final_partitions = recursive_partition(data.copy(), excluded_features, 1000, max_depth)  # Use a default threshold that makes sense for your dataset
    calculate_averages(final_partitions)
    end_time = time.time()
    times_depth.append(end_time - start_time)

# Plotting the results
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(threshold_values, times_threshold, marker='o', color='blue')
plt.title('Execution Time vs. Threshold')
plt.xlabel('Threshold')
plt.ylabel('Time (seconds)')

plt.subplot(1, 2, 2)
plt.plot(max_depth_values, times_depth, marker='o', color='green')
plt.title('Execution Time vs. Max Depth')
plt.xlabel('Max Depth')
plt.ylabel('Time (seconds)')

plt.tight_layout()
plt.show()

In [ ]:
print(threshold_values)
print(times_threshold)
print(max_depth_values)
print(times_depth)

In [ ]:
print(bics)
print(depths)
print(thresholds)

In [ ]:
print(categories)
print(bic_scores)

In [ ]:
print(best_gmm)
print(best_gmm.means_)
print(best_gmm.covariances_)